# EIN Lookup Table

In [ ]:
import pandas as pd
import psycopg2

## Number of Branches

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"
conn = psycopg2.connect(database=db_name, host = hostname) #database connection

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS ada_kcmo.ein_nb_branch AS
SELECT ein, max(nb_branches) as max_nb_branches
FROM (
    SELECT ein, year, qtr, count(*) as nb_branches
    from kcmo_lehd.mo_qcew_employers
    group by ein, year, qtr
) as a
group by ein;

alter table ada_kcmo.ein_nb_branch owner to ada_kcmo_admin;

commit;
'''
cursor = conn.cursor()
cursor.execute(query)

## Legal Name

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"
conn = psycopg2.connect(database=db_name, host = hostname) #database connection

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS ada_kcmo.ein_legal_name AS
select distinct on (ein) ein, legal_name, count
from (
    SELECT ein, legal_name, count(*) as count
    from kcmo_lehd.mo_qcew_employers
    where legal_name is not null
    group by ein, legal_name
    order by ein, count desc
) as a;

alter table ada_kcmo.ein_legal_name owner to ada_kcmo_admin;

commit;
'''
cursor = conn.cursor()
cursor.execute(query)

## NAICS

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"
conn = psycopg2.connect(database=db_name, host = hostname) #database connection

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS ada_kcmo.ein_naics AS
select distinct on (ein) ein, naics, count
from (
    SELECT ein, naics, count(*) as count
    from kcmo_lehd.mo_qcew_employers
    where naics is not null
    group by ein, naics
    order by ein, count desc
) as a;

alter table ada_kcmo.ein_naics owner to ada_kcmo_admin;

commit;
'''
cursor = conn.cursor()
cursor.execute(query)

## Zipcode

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"
conn = psycopg2.connect(database=db_name, host = hostname) #database connection

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS ada_kcmo.ein_pl_zip AS
select distinct on (ein) ein, pl_zip, count
from (
    SELECT ein, pl_zip, count(*) as count
    from kcmo_lehd.mo_qcew_employers
    where pl_zip is not null
    group by ein, pl_zip
    order by ein, count desc
) as a;

alter table ada_kcmo.ein_pl_zip owner to ada_kcmo_admin;

commit;
'''
cursor = conn.cursor()
cursor.execute(query)

## County

In [ ]:
db_name = "appliedda"
hostname = "10.10.2.10"
conn = psycopg2.connect(database=db_name, host = hostname) #database connection

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS ada_kcmo.ein_cnty AS
select distinct on (ein) ein, cnty, count
from (
    SELECT ein, cnty, count(*) as count
    from kcmo_lehd.mo_qcew_employers
    where cnty is not null
    group by ein, cnty
    order by ein, count desc
) as a;

alter table ada_kcmo.ein_cnty owner to ada_kcmo_admin;

commit;
'''
cursor = conn.cursor()
cursor.execute(query)

## Combine all tables

In [ ]:
query = '''
CREATE TABLE IF NOT EXISTS ada_kcmo.ein_lookup AS
select b.legal_name, a.*, 
    c.naics,
    case when a.max_nb_branches = 1 then d.pl_zip else null end as pl_zip,
    case when a.max_nb_branches = 1 then e.cnty else null end as cnty
from ada_kcmo.ein_nb_branch as a
left join ada_kcmo.ein_legal_name as b on a.ein = b.ein
left join ada_kcmo.ein_naics as c on a.ein = c.ein
left join ada_kcmo.ein_pl_zip as d on a.ein = d.ein 
left join ada_kcmo.ein_cnty as e on a.ein = e.ein ;

alter table ada_kcmo.ein_lookup owner to ada_kcmo_admin;

commit;
'''
cursor = conn.cursor()
cursor.execute(query)